# Transforming html to txt

<p>First, we need to extract the text from the pages. For this, we gonna use the Beautiful Soup parser in each page from local directory - the positives and negatives. We extract from paragraphs of html and append the results. Some pages has irrelevant information, like ads - we will try to remove after this.</p>
<p>After append the strings, we append the result (1 or 0), 1 if is a relevant page and 0 otherwise.</p>

In [1]:
import os
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup

In [11]:
path = "pages/positivos"

parsedPositive = []
parsed = []
y = []

for filename in os.listdir(path):
    fullpath = os.path.join(path, filename)
    print(fullpath)
    if fullpath.endswith('.html'):
        page = soup(open(fullpath),'html.parser')
        table = page.find_all('p')
        s = ""
        p = [];
        for x in table:
                if(x.text != 'placeholder'):
                    s += x.text + " ";
        s=''.join(i for i in s if  not i.isdigit())
        parsed.append(s);
        y.append(1);

pages/positivos/Maionese de Cenoura.html
pages/positivos/Receita de Cogumelo shitake na manteiga - Fácil.html
pages/positivos/receita-de-antepasto-de-berinjela-agridoce-vapt-vupt-6839.html
pages/positivos/Sagu com Molho de Soja.html
pages/positivos/Receita de Bolo de aipim vegano.html
pages/positivos/receita-de-abobora-cabotia-assada-no-forno-7103.html
pages/positivos/receita-de-hamburguer-de-grao-de-bico-facil-6971.html
pages/positivos/Quiche de Alho-poro.html
pages/positivos/Creme de Abóbora com Sálvia e Cogumelos - gordelícias.htm
pages/positivos/Receita de Molho de azeitona do Mamba | Bom Gourmet.html
pages/positivos/Charuto de folha de uva Vegano.html
pages/positivos/Falafel Frito.html
pages/positivos/Receita de Risoto de cogumelos vegano.html
pages/positivos/receita-de-arroz-de-cogumelos-facil-7072.html
pages/positivos/Receita de Quibe de assadeira vegano (berinjela).html
pages/positivos/Receita de Doce de leite vegano - com leite de amêndoas.html
pages/positivos/As Batatas _Frit

pages/positivos/Receita de Doce de banana caramelada fácil - pronto em 3 passos.html
pages/positivos/Receita de Queijo mussarela vegano.html
pages/positivos/receita-de-salgadinho-de-grao-de-bico-fit-6676.html
pages/positivos/Receita de Pure de cenoura vegano.html
pages/positivos/Berinjela Recheada com Arroz.html


In [12]:
path = "pages/negativos"

parsedNegative = []

for filename in os.listdir(path):
    fullpath = os.path.join(path, filename)
    print(fullpath)
    if fullpath.endswith('.html'):
        page = soup(open(fullpath),'html.parser')
        table = page.find_all('p')
        s = ""
        p = [];
        for x in table:
                if(x.text != 'placeholder'):
                    s += x.text + " ";
        s=''.join(i for i in s if  not i.isdigit())
        parsed.append(s);
        y.append(0)

pages/negativos/Receita de Almondegas de Aveia.html
pages/negativos/Penne Funghi ao Forno.html
pages/negativos/Receita de Caldo de Sururu.html
pages/negativos/Receita de Escondidinho de bacalhau a minha moda.html
pages/negativos/receita-de-coxa-creme-passo-a-passo-6872.html
pages/negativos/chocotone-trufado-leite-ninho-e-doce-de-leite.html
pages/negativos/receita-de-ravioli-caseiro-de-carne-7042.html
pages/negativos/Receita de Panqueca de Whey Protein.html
pages/negativos/Receita de Crepioca - Pao de queijo de frigideira.html
pages/negativos/Receita de Creme de pitaya - Fácil.html
pages/negativos/receita-de-pizza-de-brocolis-low-carb-6850.html
pages/negativos/como-fazer-chantilly.html
pages/negativos/receita-de-bife-de-carne-moida-para-hamburguer-6989.html
pages/negativos/Receita de Cozido Russo.html
pages/negativos/receita-de-torta-de-sardinha-na-frigideira-7068.html
pages/negativos/receita-de-pizza-de-tapioca-na-frigideira-6988.html
pages/negativos/receita-de-esfiha-de-carne-moida-ab

pages/negativos/Abacate Empanado.html
pages/negativos/receita-de-pao-de-frigideira-low-carb-e-facil-6900.html
pages/negativos/receita-de-frango-cremoso-light-6680.html
pages/negativos/receita-de-bolo-de-banana-com-aveia-diet-6822.html
pages/negativos/Panqueca de carne de sol.html
pages/negativos/pao-de-abobora-com-gotas-de-chocolate.html
pages/negativos/receita-de-asa-de-frango-com-molho-de-pimenta-5673.html
pages/negativos/Receita de Almondegas de tofu em molho de tomate.html
pages/negativos/Bife a Role.html
pages/negativos/Receita de Marisco vinagrete.html
pages/negativos/O negocio so nao foi fechado porque a Impossible Foods considerou a oferta.html
pages/negativos/receita-de-pudim-de-leite-condensado-sem-forno-5008.html
pages/negativos/Crespelle de Uva Itália do Chef Ravioli.html
pages/negativos/Churros de Cenoura.html
pages/negativos/receita-de-bolinho-de-arroz-integral-com-espinafre-6687.html
pages/negativos/como-fazer-kibe-frito-recheado.html
pages/negativos/receita-de-carne-de-

# Feature Extraction and Classifier
<p>After getting the parsed text and put the tag on each test, we gonna use SkLearn to make the feature extraction with Bag of Words (CountVectorizer), transforming on frequencies with TF-IDF(TfidfTransformer) and using different classifiers. We also using the pipeline to reduce the steps and be more easy to do.</p>

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

## Used Classifiers

<p>SGClassifier -> Stochastic Gradient Descent</p>
<p>MultinomialNB -> Naive Bayes</p>
<p>DecisionTreeClassifier -> Arvore de Decisao</p>
<p>LogisticRegression -> Logistic Regression</p>
<p>MLPClassifier -> Multilayer Perceptron Classifier</p>
<p>SVC -> Support Vector Classification</p>

In [5]:
skf = StratifiedKFold(n_splits=15)
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf2 = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', SGDClassifier(loss='hinge',penalty='l1', max_iter=1000, tol=None))])
text_clf3 = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', DecisionTreeClassifier())])
text_clf4 = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', LogisticRegression())])
text_clf5 = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MLPClassifier())])
text_clf6 = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', SVC())])


In [13]:
vectorizer = TfidfVectorizer()
vectorizer.fit(parsed)
print(vectorizer.vocabulary_)
print(vectorizer.idf_)

{'incorporá': 2582, 'soltas': 4509, 'extrato': 2001, 'gordura': 2384, 'container': 1122, 'dourem': 1634, 'untado': 4916, 'costela': 1198, 'gemas': 2344, 'famoso': 2032, 'flocoa': 2164, 'dificil': 1538, 'temperadas': 4693, 'pinga': 3641, 'marcante': 2962, 'relação': 4130, 'substituído': 4576, 'cozinheiro': 1238, 'orgulho': 3381, 'ágarr': 5135, 'barba': 525, 'imagina': 2541, 'escolhes': 1813, 'refrigera': 4115, 'viagem': 5039, 'abril': 18, 'tianguis': 4752, 'peneirar': 3545, 'modelável': 3111, 'caju': 711, 'pelado': 3532, 'valia': 4959, 'estiver': 1925, 'par': 3443, 'cebola': 815, 'low': 2889, 'douradinha': 1624, 'escorrida': 1819, 'duplas': 1644, 'escreva': 1821, 'esfriar': 1837, 'mexa': 3057, 'américa': 264, 'chinesa': 884, 'gasta': 2325, 'total': 4824, 'rede': 4080, 'tarte': 4676, 'acabamento': 31, 'restante': 4186, 'levando': 2822, 'exclusivos': 1974, 'cortada': 1185, 'forrei': 2211, 'passado': 3481, 'traduzido': 4837, 'segui': 4367, 'esquentar': 1895, 'muuuito': 3191, 'concordo': 10

In [14]:
acuracia_tot = []
tempo_tot = []
precisao_tot = []
recall_tot = []

for i in range(1,16):
    print ('Epoch: ', i)
    X_train, X_test, y_train, y_test = train_test_split(parsed, y, test_size=0.3, random_state=None)
    classifiers = [text_clf, text_clf2, text_clf3, text_clf4, text_clf5, text_clf6]
    acuracia = []
    tempo = []
    precisao = []
    recall = []
    for classifier in classifiers:
        time1 = time.time()
        classifier.fit(X_train,y_train)
        time2 = time.time()
        y_pred = classifier.predict(X_test)
        score = classifier.score(X_test, y_test)
        precision = precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))
        rec = recall_score(y_test, y_pred,  average='weighted', labels=np.unique(y_pred))
        acuracia.append(score)
        tempo.append(time2-time1)
        precisao.append(precision)
        recall.append(rec)
    acuracia_tot.append(acuracia)
    tempo_tot.append(tempo)
    precisao_tot.append(precisao)
    recall_tot.append(recall)

Epoch:  1
Epoch:  2
Epoch:  3
Epoch:  4
Epoch:  5
Epoch:  6
Epoch:  7
Epoch:  8
Epoch:  9
Epoch:  10
Epoch:  11
Epoch:  12
Epoch:  13
Epoch:  14
Epoch:  15


In [30]:
col = ['SGC', 'Naive Bayes', 'DecisionTree', 'LogisticRegression', 'MLP', 'SVC']
acuracia_pd = pd.DataFrame(acuracia_tot, columns = col)
acuracia_pd

,SGC,Naive Bayes,DecisionTree,LogisticRegression,MLP,SVC
0,0.771186,0.847458,0.711864,0.796610,0.838983,0.762712
1,0.762712,0.771186,0.737288,0.762712,0.779661,0.754237
2,0.669492,0.728814,0.652542,0.677966,0.703390,0.652542
3,0.661017,0.754237,0.720339,0.661017,0.779661,0.644068
4,0.728814,0.796610,0.652542,0.771186,0.788136,0.737288
5,0.754237,0.779661,0.652542,0.754237,0.754237,0.720339
6,0.669492,0.728814,0.618644,0.694915,0.762712,0.644068
7,0.677966,0.728814,0.584746,0.686441,0.737288,0.661017
8,0.711864,0.711864,0.711864,0.711864,0.737288,0.669492
9,0.694915,0.805085,0.652542,0.703390,0.771186,0.669492


In [31]:
print(np.mean(acuracia_pd))

SGC                   0.694350
Naive Bayes           0.754802
DecisionTree          0.674011
LogisticRegression    0.708475
MLP                   0.759887
SVC                   0.675706
dtype: float64


In [40]:
tempo_pd = pd.DataFrame(tempo_tot, columns = col)
tempo_pd

,SGC,Naive Bayes,DecisionTree,LogisticRegression,MLP,SVC
0,0.081759,0.286944,0.093660,0.063092,7.116157,0.163318
1,0.069931,0.275889,0.112706,0.062579,5.925161,0.155812
2,0.060302,0.294048,0.112717,0.062450,6.847748,0.152462
3,0.058402,0.276021,0.103189,0.060779,5.896733,0.145871
4,0.058971,0.289460,0.099581,0.062125,7.554471,0.158279
5,0.061379,0.277892,0.096056,0.063656,7.479907,0.163200
6,0.060149,0.281625,0.099888,0.064145,5.698795,0.150875
7,0.058135,0.272419,0.095381,0.060957,5.756792,0.147292
8,0.061413,0.295742,0.124370,0.064232,5.909900,0.158156
9,0.060178,0.294827,0.098886,0.062536,6.645585,0.151836


In [41]:
print(np.mean(tempo_pd))

SGC                   0.062049
Naive Bayes           0.284460
DecisionTree          0.103531
LogisticRegression    0.064023
MLP                   6.245184
SVC                   0.153540
dtype: float64


In [42]:
precisao_pd = pd.DataFrame(precisao_tot, columns = col)
precisao_pd

,SGC,Naive Bayes,DecisionTree,LogisticRegression,MLP,SVC
0,0.733394,0.856351,0.740664,0.780508,0.841048,0.762712
1,0.724702,0.785578,0.759916,0.728540,0.770213,0.754237
2,0.649545,0.723520,0.633475,0.661278,0.689487,0.652542
3,0.642938,0.750912,0.718903,0.642938,0.777889,0.644068
4,0.671418,0.796610,0.684566,0.750210,0.782103,0.737288
5,0.742687,0.794085,0.710678,0.742687,0.747843,0.720339
6,0.683616,0.726152,0.616628,0.700130,0.760734,0.644068
7,0.674536,0.720873,0.597400,0.694719,0.727139,0.661017
8,0.730006,0.705096,0.702257,0.707022,0.728148,0.669492
9,0.687140,0.809609,0.641712,0.702480,0.763906,0.669492


In [43]:
print(np.mean(precisao_pd))

SGC                   0.687351
Naive Bayes           0.754775
DecisionTree          0.681171
LogisticRegression    0.705481
MLP                   0.755416
SVC                   0.675706
dtype: float64


In [50]:
recall_pd = pd.DataFrame(recall_tot, columns = col)
recall_pd

,SGC,Naive Bayes,DecisionTree,LogisticRegression,MLP,SVC
0,0.771186,0.847458,0.711864,0.796610,0.838983,1.0
1,0.762712,0.771186,0.737288,0.762712,0.779661,1.0
2,0.669492,0.728814,0.652542,0.677966,0.703390,1.0
3,0.661017,0.754237,0.720339,0.661017,0.779661,1.0
4,0.728814,0.796610,0.652542,0.771186,0.788136,1.0
5,0.754237,0.779661,0.652542,0.754237,0.754237,1.0
6,0.669492,0.728814,0.618644,0.694915,0.762712,1.0
7,0.677966,0.728814,0.584746,0.686441,0.737288,1.0
8,0.711864,0.711864,0.711864,0.711864,0.737288,1.0
9,0.694915,0.805085,0.652542,0.703390,0.771186,1.0


In [45]:
print(np.mean(recall_pd))

SGC                   0.722599
Naive Bayes           0.754802
DecisionTree          0.674011
LogisticRegression    0.708475
MLP                   0.759887
SVC                   1.000000
dtype: float64


# Feature Selection

Testando selecionar as features previamentes para verificar se houve melhora no classificador


In [19]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_extraction.text import CountVectorizer

In [37]:
cv = CountVectorizer(max_df=0.9, max_features=10000)
X_vec = cv.fit_transform(X_train)
res = dict(zip(mutual_info_classif(X_vec, y_train, discrete_features=True),cv.get_feature_names()
               ))
res

{1.3518933833847613e-06: 'wok',
 2.7342850966866061e-06: 'verdade',
 4.1482190191663437e-06: 'pegue',
 5.5947872203560012e-06: 'calorias',
 7.0751327736068292e-06: 'vive',
 1.7862358426407043e-05: 'lembra',
 2.6025345344585227e-05: 'aos',
 3.8530941540611263e-05: 'seguinte',
 5.934226867598813e-05: 'opinião',
 9.4374815771014475e-05: 'cerca',
 9.4991980676181753e-05: 'cubra',
 9.9284842447547317e-05: 'compartilho',
 9.9428907308955761e-05: 'vontade',
 9.96745253764586e-05: 'hora',
 0.00010240581739409671: 'homogênea',
 0.00010455450039776465: 'bater',
 0.00010825001809720847: 'cortar',
 0.0001175109235759253: 'sonha',
 0.00012390235664328647: 'fique',
 0.0001562696627668142: 'pré',
 0.00016117476543683201: 'transparente',
 0.00019287076214060533: 'gostoso',
 0.0002041041168914931: 'temos',
 0.00020470483921997823: 'virando',
 0.0002248485325558533: 'untar',
 0.00024868752772469471: 'untada',
 0.0002551778667921889: 'servir',
 0.00027947105146554352: 'reunir',
 0.00033263978434748542: '

In [23]:
text_clf = Pipeline([('vect', cv), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf2 = Pipeline([('vect', cv), ('tfidf', TfidfTransformer()), ('clf', SGDClassifier(loss='hinge',penalty='l1', max_iter=1000, tol=None))])
text_clf3 = Pipeline([('vect', cv), ('tfidf', TfidfTransformer()), ('clf', DecisionTreeClassifier())])
text_clf4 = Pipeline([('vect', cv), ('tfidf', TfidfTransformer()), ('clf', LogisticRegression())])
text_clf5 = Pipeline([('vect', cv), ('tfidf', TfidfTransformer()), ('clf', MLPClassifier())])
text_clf6 = Pipeline([('vect', cv), ('tfidf', TfidfTransformer()), ('clf', SVC())])

In [24]:
acuracia_tot2 = []
tempo_tot2 = []
precisao_tot2 = []
recall_tot2 = []

for i in range(1,16):
    print ('Epoch: ', i)
    X_train, X_test, y_train, y_test = train_test_split(parsed, y, test_size=0.3, random_state=None)
    classifiers = [text_clf, text_clf2, text_clf3, text_clf4, text_clf5, text_clf6]
    acuracia = []
    tempo = []
    precisao = []
    recall = []
    for classifier in classifiers:
        time1 = time.time()
        classifier.fit(X_train,y_train)
        time2 = time.time()
        y_pred = classifier.predict(X_test)
        score = classifier.score(X_test, y_test)
        precision = precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))
        rec = recall_score(y_test, y_pred,  average='weighted', labels=np.unique(y_pred))
        acuracia.append(score)
        tempo.append(time2-time1)
        precisao.append(precision)
        recall.append(rec)
    acuracia_tot2.append(acuracia)
    tempo_tot2.append(tempo)
    precisao_tot2.append(precisao)
    recall_tot2.append(recall)

Epoch:  1
Epoch:  2
Epoch:  3
Epoch:  4
Epoch:  5
Epoch:  6
Epoch:  7
Epoch:  8
Epoch:  9
Epoch:  10
Epoch:  11
Epoch:  12
Epoch:  13
Epoch:  14
Epoch:  15


In [52]:
col = ['SGC', 'Naive Bayes', 'DecisionTree', 'LogisticRegression', 'MLP', 'SVC']
acuracia_pd2 = pd.DataFrame(acuracia_tot2, columns = col)
acuracia_pd2

,SGC,Naive Bayes,DecisionTree,LogisticRegression,MLP,SVC
0,0.677966,0.677966,0.669492,0.694915,0.711864,0.635593
1,0.635593,0.720339,0.601695,0.677966,0.694915,0.661017
2,0.661017,0.745763,0.661017,0.669492,0.728814,0.652542
3,0.686441,0.728814,0.618644,0.703390,0.754237,0.644068
4,0.737288,0.779661,0.720339,0.745763,0.805085,0.703390
5,0.669492,0.711864,0.686441,0.669492,0.720339,0.652542
6,0.703390,0.711864,0.644068,0.711864,0.737288,0.669492
7,0.677966,0.703390,0.652542,0.720339,0.694915,0.669492
8,0.677966,0.745763,0.703390,0.694915,0.805085,0.661017
9,0.644068,0.796610,0.627119,0.652542,0.805085,0.593220


In [53]:
print(np.mean(acuracia_pd2))

SGC                   0.689831
Naive Bayes           0.740678
DecisionTree          0.658757
LogisticRegression    0.706780
MLP                   0.749718
SVC                   0.665537
dtype: float64


In [54]:
tempo_pd2 = pd.DataFrame(tempo_tot2, columns = col)
tempo_pd2

,SGC,Naive Bayes,DecisionTree,LogisticRegression,MLP,SVC
0,0.085007,0.279718,0.118544,0.064304,6.070609,0.153494
1,0.060738,0.295795,0.101674,0.061634,6.061091,0.149521
2,0.059413,0.276343,0.090178,0.061493,5.534863,0.149536
3,0.060333,0.281294,0.092060,0.061951,6.006109,0.149856
4,0.061603,0.288482,0.128693,0.063151,6.015458,0.157222
5,0.059606,0.274176,0.103018,0.061838,5.930658,0.151713
6,0.059107,0.280988,0.095521,0.061366,5.885871,0.148424
7,0.060083,0.288757,0.110696,0.062036,6.113147,0.152964
8,0.059129,0.271849,0.083115,0.060814,5.717072,0.146601
9,0.059438,0.287933,0.099498,0.061567,5.750058,0.145534


In [55]:
print(np.mean(tempo_pd2))

SGC                   0.061559
Naive Bayes           0.283941
DecisionTree          0.104093
LogisticRegression    0.062127
MLP                   5.916104
SVC                   0.151598
dtype: float64


In [27]:
precisao_pd2 = pd.DataFrame(precisao_tot2, columns = col)
print(precisao_pd2)
print(np.mean(precisao_pd2))

         SGC  Naive Bayes  DecisionTree  LogisticRegression       MLP  \
0   0.736077     0.677966      0.678622            0.731780  0.702982   
1   0.571056     0.734721      0.619063            0.651851  0.688117   
2   0.630056     0.739356      0.665001            0.649545  0.719198   
3   0.714093     0.726152      0.601269            0.704007  0.748458   
4   0.728657     0.783563      0.713743            0.730404  0.798023   
5   0.656144     0.699571      0.669115            0.649545  0.710632   
6   0.702480     0.705096      0.635059            0.707022  0.725747   
7   0.643290     0.689407      0.629533            0.718884  0.674605   
8   0.700714     0.738712      0.690692            0.736935  0.806475   
9   0.777542     0.801984      0.615115            0.709820  0.824262   
10  0.711963     0.806867      0.649153            0.746610  0.827909   
11  0.741615     0.724499      0.702547            0.752584  0.759026   
12  0.741615     0.771075      0.676152            

In [28]:
recall_pd2 = pd.DataFrame(recall_tot2, columns = col)
print(recall_pd2)
print(np.mean(recall_pd2))

         SGC  Naive Bayes  DecisionTree  LogisticRegression       MLP  SVC
0   0.677966     0.677966      0.669492            0.694915  0.711864  1.0
1   0.635593     0.720339      0.601695            0.677966  0.694915  1.0
2   0.661017     0.745763      0.661017            0.669492  0.728814  1.0
3   0.686441     0.728814      0.618644            0.703390  0.754237  1.0
4   0.737288     0.779661      0.720339            0.745763  0.805085  1.0
5   0.669492     0.711864      0.686441            0.669492  0.720339  1.0
6   0.703390     0.711864      0.644068            0.711864  0.737288  1.0
7   0.677966     0.703390      0.652542            0.720339  0.694915  1.0
8   0.677966     0.745763      0.703390            0.694915  0.805085  1.0
9   0.644068     0.796610      0.627119            0.652542  0.805085  1.0
10  0.737288     0.805085      0.661017            0.762712  0.830508  1.0
11  0.745763     0.728814      0.720339            0.762712  0.762712  1.0
12  0.745763     0.754237

In [29]:
print(np.mean(acuracia_pd2) - np.mean(acuracia_pd))

SGC                  -0.004520
Naive Bayes          -0.014124
DecisionTree         -0.015254
LogisticRegression   -0.001695
MLP                  -0.010169
SVC                  -0.010169
dtype: float64
